## Import Library yang digunakan

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import  StringLookup, CategoryEncoding, Layer
import numpy as np
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("DataDestinasi.csv")
df.head(5)

,Place_Id,Place_Name,Description,Unnamed: 3,City,Price,Rating,Alamat Detail,Category
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,NaN,Jakarta,20000,4.6,"Jl. Medan Merdeka Barat, Gambir, Jakarta Pusat",Sejarah dan Budaya
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",NaN,Jakarta,0,4.6,"Jl. Taman Fatahillah, Pinangsia, Jakarta Barat",Sejarah dan Budaya
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,NaN,Jakarta,270000,4.6,"Jl. Lodan Timur No.7, Ancol, Jakarta Utara",Taman Hiburan
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,NaN,Jakarta,10000,4.5,"Jl. Taman Mini Indonesia Indah, Cipayung, Jaka...",Sejarah dan Budaya
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,NaN,Jakarta,94000,4.5,"Jl. Lodan Timur No.7, Ancol, Jakarta Utara",Taman Air


## Melakukan data cleaning dan data preprataion

In [ ]:
df['Price'] = pd.to_numeric(df['Price'], errors='coerce').fillna(0).astype(int)

print("Jumlah nilai 0 pada kolom 'Price' sebelum diganti:", (df['Price'] < 5000).sum())

replacement_values = [5000, 10000, 15000, 20000]

df['Price'] = df['Price'].apply(lambda x: np.random.choice(replacement_values) if x < 5000 else x)

print("Jumlah nilai 0 pada kolom 'Price' setelah diganti:", (df['Price'] < 5000).sum())

## Pemodelan arsitektur machine learning


1.   Normalization column dataset
2.   Arsitektur Model
3.   Modeling
4.   Save Model





In [ ]:
# Normalizer untuk harga
price_normalizer = layers.Normalization()
price_normalizer.adapt(df['Price'].values.reshape(-1, 1))

# Normalizer untuk rating
rating_normalizer = layers.Normalization()
rating_normalizer.adapt(df['Rating'].values.reshape(-1, 1))
# Input untuk rating
rating_input = layers.Input(shape=(1,), dtype=tf.float32, name="rating")
rating_scaled = rating_normalizer(rating_input)
# String lookup untuk kota
unique_cities = df['City'].unique()
city_lookup = layers.StringLookup(vocabulary=unique_cities)

# Category encoding untuk kota
city_encoder = layers.CategoryEncoding(num_tokens=len(unique_cities) + 1, output_mode="one_hot")

# String lookup untuk kategori
unique_categories = df['Category'].unique()
category_lookup = StringLookup(vocabulary=unique_categories)

# Category encoding untuk kategori
category_encoder = CategoryEncoding(num_tokens=len(unique_categories) + 1, output_mode="one_hot")


# Label encoding untuk tempat
label_encoder = {place: idx for idx, place in enumerate(df['Place_Name'].unique())}
df['Place_Label'] = df['Place_Name'].map(label_encoder)

# Input untuk model
price_input = layers.Input(shape=(1,), dtype=tf.float32, name="price")
city_input = layers.Input(shape=(1,), dtype=tf.string, name="city")
category_input = layers.Input(shape=(1,), dtype=tf.string, name="category")

# Preprocessing di dalam model
price_scaled = price_normalizer(price_input)
city_index = city_lookup(city_input)
city_one_hot = city_encoder(city_index)
category_index = category_lookup(category_input)
category_one_hot = category_encoder(category_index)

# Menggabungkan preprocessing
preprocessed_input = layers.Concatenate()([price_scaled, city_one_hot, rating_scaled, category_one_hot])

# Arsitektur model
x = layers.Dense(128, activation='relu')(preprocessed_input)
x = layers.Dense(64, activation='relu')(x)
output = layers.Dense(len(label_encoder), activation='softmax')(x)


model = Model(inputs=[price_input, city_input, rating_input, category_input], outputs=output)

# Kompilasi model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data latih (mengonversi 'Price' menjadi float)
train_prices = df['Price'].values.astype(float)
train_cities = df['City'].values
train_ratings = df['Rating'].values.astype(float)
train_categories = df['Category'].values
train_labels = df['Place_Label'].values

# Latih model
model.fit([train_prices, train_cities, train_ratings, train_categories], train_labels, epochs=300, batch_size=32)

# Simpan model
model.save("model_destinasi_wisata_with_preprocessing.h5")


Epoch 1/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.0000e+00 - loss: 6.3396
Epoch 2/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0153 - loss: 6.3066
Epoch 3/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0386 - loss: 6.2518
Epoch 4/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0489 - loss: 6.1398
Epoch 5/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0549 - loss: 5.9396
Epoch 6/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0778 - loss: 5.6345
Epoch 7/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0834 - loss: 5.1585
Epoch 8/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1010 - loss: 4.6189
Epoch 9/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1763 - loss: 4.1030
Epoch 10/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2055 - loss: 3.5862
Epoch 11/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2411 - loss: 3.1846
Epoch 12/300
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accur

## Pembuatan function untuk melakukan prediksi hasil

In [ ]:
def recommend_places(price, city, rating, category, top_n=5):
    # Filter data sesuai dengan kota
    city_filtered = df[df['City'] == city]

    # Jika tidak ada data sesuai kota, kembalikan pesan
    if city_filtered.empty:
        return f"No places found for city: {city}"

    # Ambil indeks dari data yang difilter
    filtered_indices = city_filtered.index.tolist()
    filtered_places_info = city_filtered[['Place_Name', 'City', 'Price', 'Category']].to_numpy()

    # Konversi input
    price_input = tf.constant([price], dtype=tf.float32)
    city_input = tf.constant([city], dtype=tf.string)
    rating_input = tf.constant([rating], dtype=tf.float32)
    category_input = tf.constant([category], dtype=tf.string)

    # Prediksi hanya untuk data yang difilter
    probabilities = model.predict([price_input, city_input, rating_input, category_input])[0]

    # Ambil prediksi untuk data yang difilter
    filtered_probabilities = probabilities[filtered_indices]
    top_indices = np.argsort(filtered_probabilities)[-30:][::-1]
    # Ambil informasi tempat langsung dari filtered_places_info
    recommendations = []
    for idx in top_indices:
        place_name, place_city, place_price, place_category = filtered_places_info[idx]
        if place_city == city and place_price < price:
          recommendations.append({
              "Place_Name": place_name,
              "City": place_city,
              "Price": int(place_price),
              "Category": place_category
          })
        if len(recommendations) == top_n:
          break

    return recommendations

# Prediksi dengan fitur tambahan kategori
result = recommend_places(30000, "Bandung",4, "Nature", top_n=3)
for i in result:
    print(i)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
{'Place_Name': 'Kampoeng Tulip', 'City': 'Bandung', 'Price': 15000, 'Category': 'Nature'}
{'Place_Name': 'Bukit Jamur', 'City': 'Bandung', 'Price': 20000, 'Category': 'Nature'}
{'Place_Name': 'Kebun Binatang Bandung', 'City': 'Bandung', 'Price': 20000, 'Category': 'Adventure'}
